In [ ]:
# 題：缺陷分類模型
# 使用 TensorFlow 來構建一個卷積神經網絡（CNN）來進行缺陷圖像的分類。
# 1. 數據準備
# 使用 ImageDataGenerator 進行數據增強和預處理。

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 配置圖像數據增強
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) ##
# rescale 用於縮放圖像的像素值。
# 圖像的像素值通常在 0 到 255 之間，rescale=1./255 表示將像素值縮放到 0 到 1 之間 (即除以 255)，這樣有助於加快訓練過程並提高模型的收斂效果。
# validation_split 用於將數據集拆分為訓練集和驗證集，此的 0.2 表示將 20% 的數據用作驗證集，其餘 80% 用於訓練集。

# 構建訓練集和驗證集生成器
train_generator = datagen.flow_from_directory( ##
    'data/train',  # 圖像數據的文件夾路徑
    target_size=(128, 128),  # 調整圖像大小
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'data/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# ImageDataGenerator 用來從文件夾中批量載入圖像並自動進行增強處理。
# target_size 用於設置將所有圖像調整到的目標大小。單位是 px，表示寬度和高度。
# batch_size 指定每次生成的圖像批次大小，此的 32 表示每次生成 32 張圖像作為一個批次。較小的 batch_size 會讓模型更新更頻繁，但速度較慢；較大的 batch_size 可以加快訓練速度，但可能需要更多的內存。
# class_mode 指定標籤的類型。
    # binary：用於二元分類，輸出為 0 或 1，適合兩分類問題。
    # categorical：用於多分類問題，會輸出 one-hot 編碼的向量，適合三類或三類以上的分類。
    # sparse：適合多分類問題，標籤是整數而不是 one-hot 編碼。
# subset 用於指定數據集的子集。
    # training：表示加載訓練集。
    # validation：表示加載驗證集。

In [ ]:
# 2. 構建卷積神經網絡
# 使用 TensorFlow 的 Sequential API 構建簡單的 CNN 模型。

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 初始化模型
model = Sequential([ ##
    # 第一個卷積層
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    # 第一個池化層 (用來減小圖像的尺寸，降低計算量並增強特徵的魯棒性。但不會改變深度。)
    MaxPooling2D((2, 2)),
    # 第二個卷積層 (用來提取更高層次的特徵)
    Conv2D(64, (3, 3), activation='relu'),
    # 第二個池化層 (繼續減小圖像尺寸)
    MaxPooling2D((2, 2)),
    # 展平層 (一維向量)
    Flatten(),
    # 全連接層 (用於學習圖像的高階特徵，開始預測)
    Dense(64, activation='relu'),
    # 輸出層
    Dense(1, activation='sigmoid')  # 二元分類，輸出一個 0 或 1
])

# 32：表示這層有 32 個卷積核（filters），即這一層會提取 32 種不同的特徵，所以深度從 3 (通道數) 變成 32。即輸出尺寸 (128, 128, 3) 變成 (128, 128, 32)。
# (3, 3)：卷積核的大小為 3x3 px。
# activation='relu'：使用 ReLU 作為激活函數，可以引入非線性特徵。
# input_shape=(128, 128, 3)：輸入圖像的形狀為 128x128 px，並有 3 個顏色通道（RGB）。

# (2, 2)：池化窗口大小為 2x2，會將輸入特徵圖的尺寸縮小一半。
# 輸出尺寸：從 (128, 128, 32) 減小到 (64, 64, 32)。

# 64：表示這層有 64 個卷積核，這一層會提取 64 種不同的特徵。即輸出尺寸變成 (64, 64, 64)。
# 注意，這一層不需要指定 input_shape，因為模型已經知道前一層的輸出形狀。

# (2, 2)：池化窗口大小為 2x2，將輸入特徵圖的尺寸再次縮小一半。
# 輸出尺寸變成 (32, 32, 64)。

# 假設通過前面的卷積層和池化層後，特徵圖的尺寸是 (32, 32, 64)，那麼 Flatten() 會將其展平為長度為 32*32*64 的一維向量。
# 輸出尺寸：32 x 32 x 64 = 65536，即長度為 65536 的一維向量。

# 64：全連接層中的神經元數量。
# 輸出尺寸：將 65536 維的向量映射到 64 維的向量。

# 1：輸出層的神經元數量為 1，表示輸出一個單一的值。
# activation='sigmoid'：使用 Sigmoid 激活函數，將輸出壓縮到 0 和 1 之間，用於二元分類（例如有缺陷和無缺陷）。
# 輸出尺寸：1 維，用於二元分類，表示二元分類的概率值（介於 0 和 1 之間）。


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# 3. 編譯與訓練模型
# 編譯模型並使用訓練集進行訓練。

# 配置模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) ##
# compile 方法用來配置模型的訓練設置。
# optimizer='adam'：指定使用 Adam 優化器。
# Adam 是一種自適應的優化算法，結合了動量和學習率調整，能夠加速收斂並在很多情況下取得良好的結果。

# loss='binary_crossentropy'：指定損失函數為二元交叉熵。
# 二元交叉熵適用於二分類問題（即兩個類別，例如「有缺陷」和「無缺陷」）。
# 損失函數的作用是衡量模型的預測結果與真實標籤之間的誤差。模型會通過優化這個損失函數來調整內部參數，以使預測更準確。

# metrics=['accuracy']：指定評估指標為準確率（accuracy）。
# 

# 訓練模型
model.fit(train_generator, validation_data=validation_generator, epochs=10)
# fit 方法用來訓練模型。它會使用訓練數據來更新模型的參數，使得模型能夠學習數據中的模式。
# train_generator 是一種圖像數據生成器，它會批量地將訓練圖像提供給模型，並進行數據增強操作。

# validation_data=validation_generator：驗證數據生成器。
# 驗證數據用來在訓練過程中評估模型的性能，以便在訓練過程中監控模型是否過擬合或欠擬合。
# 每個 epoch 結束後，模型會在驗證數據上評估其性能，從而可以觀察訓練集和驗證集的準確率差異。

# epochs=10：訓練迭代次數。
# 在一個 epoch 中，模型會看到整個訓練數據一次。


In [ ]:
# 4. 保存模型
# 將訓練好的模型保存到文件中。

model.save('defect_classification_model.h5')
# model.save() 主要用於深度學習模型 (如 Keras 或 TensorFlow) 專門設計的儲存。
